<center><img src="../images/DLI Header.png" alt="Header" style="width: 400px;"/></center>

In [1]:
### Run this code to initialize model functions

In [1]:
import torch
import torchvision
train_loader = torch.utils.data.DataLoader( torchvision.datasets.MNIST('/files/', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])),batch_size=60000, shuffle=True)

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import PIL.Image as Image

In [3]:
training_data = next(enumerate(train_loader))[1][0]
std,mean=torch.std_mean(training_data)

In [4]:
train_loader = torch.utils.data.DataLoader( torchvision.datasets.MNIST('/files/', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((mean,),(std,))])),batch_size=64, shuffle=True)

In [5]:
test_loader = torch.utils.data.DataLoader( torchvision.datasets.MNIST('/files/', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((mean,),(std,))])),batch_size=1000, shuffle=True)

In [6]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.Conv1=nn.Conv2d(1,10,kernel_size=5)
    self.Conv2=nn.Conv2d(10,20,kernel_size=5)
    self.ConvDrop=nn.Dropout2d()
    self.Full1=nn.Linear(320,50)
    self.Full2=nn.Linear(50,10)
  def forward(self,x):
    x=self.Conv1(x)
    x = F.max_pool2d(x, 2)
    x=F.relu(x)
    x=self.Conv2(x)
    x=F.relu(x)
    x=self.ConvDrop(x)
    x = F.max_pool2d(x, 2)
    x=F.relu(x)
    x = x.view(-1, 320)
    x=self.Full1(x)
    x=F.relu(x)
    x=self.Full2(x)
    x=F.log_softmax(x)
    return x

In [7]:
network = Net()
learning_rate = 0.01
momentum = 0.5
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    count=True
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [9]:
n_epochs = 3
log_interval = 10
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'results/model.pth')
      torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
# Run this code the first time, to initialize and train the model with the MNIST dataset

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


### Camera

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;">
    <h4><i>Tip</i></h4>
    <p>There can only be one instance of CSICamera or USBCamera at a time.  Before starting this notebook, make sure you have executed the final "shutdown" cell in any other notebooks you have run so that the camera is released. 
    </p>
</div>

In [10]:
# Check device number
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Jul 19 00:03 /dev/video0


In [11]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


In [12]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'numbers'

CATEGORIES = ['0','1', '2', '3', '4', '5', '6', '7', '8', '9']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('./data/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

numbers task with ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] categories defined


In [13]:
# Set up the data directory location if not there already
DATA_DIR = '/nvdli-nano/project/data'
!mkdir -p {DATA_DIR}

### Data Collection

In [14]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import numpy

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    img = camera.value
    img = PIL.Image.fromarray(img)
    img = torchvision.transforms.functional.rgb_to_grayscale(img,1)
    img = torchvision.transforms.Resize((28,28))(img)
    img = torchvision.transforms.functional.invert(img)
    img= torchvision.transforms.functional.adjust_contrast(img,10)
    img = numpy.array(img)
    dataset.save_entry(img, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

data_collection_widget created


### Model

In [15]:
import torch
import torchvision


device = torch.device('cuda')

# model = model.to(device)
network=Net()
checkpoint=torch.load('results/model.pth')
network.load_state_dict(checkpoint)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='/nvdli-nano/project/results/my_model.pth')

network.cuda()

def load_model(c):
    network.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(network.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

# display(model_widget)
print("model configured and model_widget created")

model configured and model_widget created


### Live  Execution

In [16]:
import threading
import time
import torch.nn.functional as F
import PIL.Image

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
mean=torch.Tensor([0.485]).cuda()
std = torch.Tensor([0.229]).cuda()
def live(state_widget, model, camera, prediction_widget):
    global dataset
    count=0
    while state_widget.value == 'live':
        image = camera.value
        image = PIL.Image.fromarray(image)
        image = torchvision.transforms.functional.rgb_to_grayscale(image,1)
        image = torchvision.transforms.Resize((28,28))(image)
        image = torchvision.transforms.functional.invert(image)
        image= torchvision.transforms.functional.adjust_contrast(image,10)
        image = torchvision.transforms.functional.to_tensor(image).to(device)
        image.sub(mean[:]).div_(std[:])
        image = image[None, ...]
        output = network(image)
        category_index = output.data.max(1, keepdim=True)[1]
        prediction_widget.value = dataset.categories[category_index]
        time.sleep(0.25)

def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, network, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")
state_widget.value = 'live'

live_execution_widget created


### Training and Evaluation

In [19]:
epochs_widget = ipywidgets.IntText(description='epochs', value=1)
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')

BATCH_SIZE = 8
log_interval = 2
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(epochs_widget.value + 1)]

def train_eval(is_training):
    global BATCH_SIZE, learning_rate, momentum, network, dataset, optimizer, train_button, loss_widget, state_widget, epochs_widget
    train_loader = torch.utils.data.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True)
    state_widget.value = 'stop'
    train_button.disabled = True
    time.sleep(1)
    if is_training:
        network = network.train()
    else:
        network = network.eval()
    while epochs_widget.value > 0:
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = network(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                loss_widget.value = loss.item()
                train_losses.append(loss.item())
                train_counter.append((batch_idx*64) + ((epochs_widget.value-1)*len(train_loader.dataset)))
                torch.save(network.state_dict(), 'results/model.pth')
                torch.save(optimizer.state_dict(), 'results/optimizer.pth')
        epochs_widget.value-=1
    train_button.disabled = False
    state_widget.value = 'live'
train_button.on_click(lambda c: train_eval(is_training=True))

train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    loss_widget,
    ipywidgets.HBox([train_button])
])
print("training widget created")

training widget created


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Display the Interactive Tool!

The interactive tool includes widgets for data collection, training, and testing.

In [20]:
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [56]:
image = camera.value
image=PIL.Image.fromarray(image)
print(type(image))

<class 'PIL.Image.Image'>


<h1 style="background-color:#76b900;"></h1>

## Before you go...<br><br>Shut down the camera and/or notebook kernel to release the camera resource

In [ ]:
# Attention!  Execute this cell before moving to another notebook
# The USB camera application only requires that the notebook be reset
# The CSI camera application requires that the 'camera' object be specifically released

import os
import IPython

if type(camera) is CSICamera:
    print("Ignore 'Exception in thread' tracebacks\n")
    camera.cap.release()

os._exit(00)

Return to the DLI course pages for the next instructions.

<center><img src="../images/DLI Header.png" alt="Header" style="width: 400px;"/></center>